# LangChain RAG App 🦜

Based on our problem statement, we need to build a customer service chatbot that can answer questions based on our product documentation.

## Load Libraries

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_core.messages import AIMessage
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## Craft our Prompt

In our approach, we'll utilize a prompt that incorporates a MessagesPlaceholder variable, labeled as "chat_history." This enables us to input a list of Messages into the prompt via the "chat_history" input key. These messages will be placed between the system message and the latest human message containing the current question.

In [ ]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest customer question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## Initialize GPT4o

In [ ]:
model = AzureChatOpenAI(
    azure_deployment="gpt4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

## Initialize Retriever

A retriever is an interface that returns documents given an unstructured query. In our case, we want to retrieve product information from Azure Search.

In [ ]:
retriever = AzureAISearchRetriever(
    content_key="content", top_k=5, index_name="products", api_key=os.getenv("AZURE_SEARCH_KEY"), service_name="genai-on-azure-search"
)

## Initialize the Chat History Chain

Instantiate a chain that is aware of chat history. The process involves adding a rephrased version of the input query to the beginning of the information sent to our Azure Search. By doing this, the retriever can better understand and incorporate the context of the ongoing conversation when retrieving relevant information.

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

## Initialize the RAG Chain

We will use create_stuff_documents_chain to develop a question_answer_chain. This chain takes in the input keys: context, chat_history, and input. It uses the retrieved context, the conversation history, and the query to generate an answer. Since it stuff's all documents retrieved into the prompt, it is very important to ensure they fit into the context window (ie: token limits). In our case, we have small enough chunks to not have to worry about that.

We construct our final rag_chain using create_retrieval_chain. This chain sequentially applies the history_aware_retriever and question_answer_chain, retaining intermediate outputs like the retrieved context for convenience. It takes input and chat_history as input keys and produces input, chat_history, context, and answer in its output.

In [ ]:
system_prompt = (
    "You are an customer service assistant for question-answering tasks on our companies product information. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## Adding Chat History

Generally speaking, to manage chat history, we need two things:

1. An objtect for storing the chat history

2. An object that wraps our chain and manages updates to the chat history


We will use BaseChatMessageHistory and RunnableWithMessageHistory for these tasks. RunnableWithMessageHistory is a wrapper for an LCEL chain and a BaseChatMessageHistory, managing the injection of chat history into inputs and updating it after each use.

Below, we demonstrate a simple example of the second option, where chat histories are stored in a simple dictionary. LangChain also supports memory integrations with CosmosDB and other technologies for more robust persistence.

RunnableWithMessageHistory instances manage the chat history for you. They accept a configuration with a key (default is "session_id") that specifies which conversation history to fetch and prepend to the input, then append the output to the same conversation history.

In [ ]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## Question #1

In [ ]:
conversational_rag_chain.invoke(
    {"input": "How much is the home theater system?"},
    config={
        "configurable": {"session_id": "session1"}
    }, 
)["answer"]

## Follow Up Question

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What does it include?"},
    config={"configurable": {"session_id": "session1"}},
)["answer"]

## Inspect Conversation History

In [ ]:
for message in store["session1"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")